In [1]:
import sys
import os
import time
import tweepy
import pandas as pd
import nltk
import numpy as np
from tweepy import OAuthHandler
from tweepy import Cursor
from tweepy import Paginator

#creazione dizionario dei brands
brands_dict = {}
brands_dict["alfaromeo"] = 'AlfaRomeoUSA'
brands_dict["samsung"] = 'SamsungUS'
brands_dict["lavazza"] = 'lavazzaUSA'
brands_dict["armani"] = 'armani'
brands_dict["adidas"] = 'adidasUS'
brands_dict["microsoft"] = 'Microsoft'
brands_dict["barilla"] = 'BarillaUS'
brands_dict["ferrero"] = 'FerreroUK'
brands_dict["delonghi"] = 'DeLonghiUK'
brands_dict["amarelli"] = 'MuseoAmarelli'
all_brands_list = ['alfaromeo', 'samsung', 'lavazza', 'armani', 'adidas', 'microsoft', 'barilla', 'ferrero', 'delonghi','amarelli']

#CHIAVI API TWITTER 
cons_key = '7a1egZomKUUFJZd0fsLZTofTq'
cons_secret = 'iPlTUWVDi3A6tt2BF6D0tDuYPjJ6yUzaMEzXEAw6dpkoSsXD1o'
acc_token = '1497898238795472900-oqEVmk9x5UuMX61KzxTj5Jay6Rn4bV'
acc_secret = 'xbGTCGvpQu8yl0M7c1qSWi5RY1w9rV2wMQzb2UrqhRCwP'
bearer = 'AAAAAAAAAAAAAAAAAAAAAEcpjAEAAAAA8sTuDav%2BYe6qPyGGOjB6cv%2FtUbE%3DkqVbW8FVTHYSiXsiobUfgm9mU9EkLC8lbPv6El606gD3lfYOXa'
client_server = '5xi46p9-pr7x0IjnAWU_dxb2Bz_UJNPo3PUfDMvfGEN4NP8nHG'

#Funzione che restituisce l'autenticazione a Twitter
def get_twitter_auth():
    try:
        consumer_key = cons_key
        consumer_secret = cons_secret
        access_token = acc_token
        access_secret = acc_secret
    except KeyError:
        sys.stderr.write("Twitter Environment Variable not Set\n")
        sys.exit(1)     
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    return auth


#Funzione che restituisce il client per accedere all'API di autenticazione
def get_twitter_client():
    auth = get_twitter_auth()
    client = tweepy.API(auth, wait_on_rate_limit=True) 
    return client


#controlla se l'utente segue la pagina:
#restituisce un booleano
client = get_twitter_client()
def check_friendship(source_screen_name, target_screen_name):
    try:
        friendship = client.get_friendship(source_screen_name = source_screen_name, target_screen_name = target_screen_name)
        return friendship[0].following
    except:
        return False



#lista degli utenti USERS-TAGS-VALUES
users_tags_df = pd.read_csv('final_results/users_tags_values.csv')
all_users_list = list(np.unique(users_tags_df["User"])) #lista di utenti #lista di utenti
print(str(len(all_users_list))+" users")

498 users


In [ ]:
#GOLD STANDARD 

#triade di valori <User , Brand , boolean>, sul quale per coppia (Utente,Brand) attribuiamo
# il parametro boolean per indicare se l'utente segue o meno la pagina ufficiale del brand.
gold_standard = pd.DataFrame(columns=['User','Brand','boolean'])

#riempimento del dataframe con i valori riportati da check_friendship
#per tutti gli utenti
for user in all_users_list:
    print("Check for "+user+":")
    #per tutte le pagine dei brands
    for brand in all_brands_list:
        brand_page = brands_dict.get(brand)
        #se l'utente segue la pagina:
        if check_friendship(user,brand_page):
            #assegna True
            gold_standard.loc[len(gold_standard)] = [user,brand,True]
        #se l'utente NON segue la pagina:
        else:
            #assegna False
            gold_standard.loc[len(gold_standard)] = [user,brand,False]

#salvataggio:
if not os.path.exists('final_results'):
    os.makedirs('final_results')
gold_standard.to_csv("final_results/gold_standard_df.csv", index=False) 

In [2]:
#TRUE GOLD STANDARD: Lista degli elementi del GOLD STANDARD per i soli utenti con boolean True

true_gold_standard = pd.read_csv('final_results/gold_standard_df.csv')
true_gold_standard.drop(true_gold_standard[true_gold_standard.boolean == False].index, inplace=True)
true_gold_standard = true_gold_standard.drop('boolean',axis=1).reset_index(drop=True)
true_gold_standard.to_excel('final_results/true_gold_standard.xlsx',index=False)
true_gold_standard

,User,Brand
0,007maybe,ferrero
1,21Phylk,ferrero
2,ABO_mondo,amarelli
3,ACE2276,alfaromeo
4,AGnomblea,ferrero
...,...,...
618,wittystwitty,armani
619,wittystwitty,microsoft
620,wittystwitty,ferrero
621,xkj1985x,samsung


In [3]:
#CREAZIONE DELLO STRONG_GOLD STANDARD:
# (lista di utenti presenti sul true_gold_standard) INTERSEZIONE (lista di utenti in cui la sentiment analysis ha dato un giudizio positivo per i brand)

true_gold_standard = pd.read_excel('final_results/true_gold_standard.xlsx') #true_gold_standard
true_users_list = np.unique(true_gold_standard['User'].to_list())           #lista degli utenti in true_gold_standard
true_GS_dict = dict(tuple(true_gold_standard.groupby('User')))              #raggruppa gli utenti del true gold standard per User

users_brands_df = pd.read_csv('final_results/users_brands_values.csv')      #tabella USERS-BRANDS VALUES


def get_user_class(user):
    return true_GS_dict.get(user)['Brand'].to_list()

#restituisce il sentiment di un utente rispetto al brand
def get_user_brand_sentiment(user,brand):
    return users_brands_df.loc[users_brands_df['User'] == user,brand].values[0]

#creazione del dataframe per strong_gold_standard
strong_gold_standard = pd.DataFrame(columns=['User','Brand'])
for user in true_users_list:
    classes = get_user_class(user)
    for brand in classes:
        if get_user_brand_sentiment(user,brand) >0.67:
            strong_gold_standard.loc[len(strong_gold_standard)] = [user,brand]

strong_users_list = strong_gold_standard['User'].to_list()
strong_gold_standard.to_excel('final_results/strong_gold_standard.xlsx',index=False)
strong_gold_standard

,User,Brand
0,AnteaterButters,barilla
1,AshleyUGCsocial,lavazza
2,CharliCass,ferrero
3,CoriglianoCala1,amarelli
4,Dawnr1979,delonghi
5,DeniseMorbin,delonghi
6,DigitalVTajc,microsoft
7,DitchMariapaola,delonghi
8,DuranWebDev,lavazza
9,FirstMediaTeam,microsoft


In [4]:
#CREAZIONE DELLO SENTIMENT_GOLD STANDARD:
#lista di utenti in cui la sentiment analysis ha dato un giudizio positivo per i brand
users_brands_df = pd.read_csv('final_results/users_brands_values.csv')  #tabella USERS-BRANDS VALUES
all_users_list = list(np.unique(users_brands_df["User"]))               #lista di utenti
all_brands_list = ['alfaromeo', 'samsung', 'lavazza', 'armani', 'adidas', 'microsoft', 'barilla', 'ferrero', 'delonghi','amarelli']

#restituisce il sentiment di un utente rispetto al brand
def get_user_brand_sentiment(user,brand):
    return users_brands_df.loc[users_brands_df['User'] == user,brand].values[0]

#creazione del dataframe sentiment_gold_standard
sentiment_gold_standard = pd.DataFrame(columns=['User','Brand'])
for user in all_users_list:
    for brand in all_brands_list:
        if get_user_brand_sentiment(user,brand) >0.67:
            sentiment_gold_standard.loc[len(sentiment_gold_standard)] = [user,brand]


sentiment_gold_standard.to_excel('final_results/sentiment_gold_standard.xlsx',index=False)
sentiment_gold_standard

,User,Brand
0,007maybe,alfaromeo
1,21Phylk,armani
2,21Phylk,adidas
3,ABO_mondo,alfaromeo
4,ABO_mondo,armani
...,...,...
326,thesweetpaisana,ferrero
327,tuscanysusie,alfaromeo
328,tuscanysusie,barilla
329,weetish,ferrero
